- depth + bagging_temperature + 더 깊게 학습하게 만들기.
- 해볼 것
-> bagging_temperature 이란?
-> 더 깊게 학습하는 방법은?
-> overtfittign 을 막을 수 있는 방법은?

---
- model 5개의 평균을 만들어서 평가하고 있는데, 그 중 가장 좋은 model로 값을 뽑아내면 좋지 않을까?

- threshold 를 for 문 구간을 통해서 구하게 변경

- 산술평균 앙상블 대신에, threshold 값을 기준으로 하드보팅 앙상블 방법으로 진행해보는 것은 어떨까?
-> threshold 인 pred_proba가 0.4 이상인 값이 3개 이상이라면 True 같은 느낌.

In [146]:
DATA_PATH = "data/"
SUBMIT_PATH = "submit/"
SEED = 42

In [147]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier
from datetime import datetime

In [148]:
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')


train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [149]:
# hour 변수를 넣기 위해서 추가
train_data['contents_open_dt'] = train_data['contents_open_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
test_data['contents_open_dt'] = test_data['contents_open_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

train_data['hour'] = train_data['contents_open_dt'].apply(lambda x : x.timetuple()[3])
test_data['hour'] = test_data['contents_open_dt'].apply(lambda x : x.timetuple()[3])

train_data.head(3)

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,...,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target,hour
0,0,True,True,True,False,False,False,1,4,3,...,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1,12
1,1,False,False,False,True,True,False,1,3,4,...,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0,17
2,2,False,False,False,True,False,False,2,0,3,...,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0,20


In [150]:
# Train data EDA에 따라서
# good - mid - low time (target에 따라서 good <-> low 범위)
# low : 23~07
# mid : 17~22
# good : 8~16
train_data.loc[train_data['hour']<=7, 'hour_band'] = 1 # 'low_time'
train_data.loc[train_data['hour']==23, 'hour_band'] = 1 #'low_time'

train_data.loc[(train_data['hour']>7) & (train_data['hour'] <= 16), 'hour_band'] = 3 # 'good_time'
train_data.loc[(train_data['hour']>=17) & (train_data['hour']<=22), 'hour_band'] = 2#'mid_time'
train_data = train_data.drop(columns = ['hour'])

test_data.loc[test_data['hour']<=7, 'hour_band'] = 1#'low_time'
test_data.loc[test_data['hour']==23, 'hour_band'] = 1#'low_time'

test_data.loc[(test_data['hour']>7) & (test_data['hour'] <= 16), 'hour_band'] = 3#'good_time'
test_data.loc[(test_data['hour']>=17) & (test_data['hour']<=22), 'hour_band'] = 2#'mid_time'
test_data = test_data.drop(columns = ['hour'])

In [151]:
from typing import Dict
import numpy as np
import pandas as pd

def add_code(
    df: pd.DataFrame,
    d_code: Dict[int, Dict[str, int]], 
    h_code: Dict[int, Dict[str, int]], 
    l_code: Dict[int, Dict[str, int]],
) -> pd.DataFrame:
    
    # Copy input data
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df

In [152]:
d_code = pd.read_csv('data/속성_D_코드.csv', index_col=0).T.to_dict()
h_code = pd.read_csv('data/속성_H_코드.csv', index_col=0).T.to_dict()
l_code = pd.read_csv('data/속성_L_코드.csv', index_col=0).T.to_dict()


train_data = add_code(train_data, d_code, h_code, l_code)
test_data = add_code(test_data, d_code, h_code, l_code)

In [153]:
train_data.columns

Index(['id', 'd_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_f', 'person_prefer_g',
       'person_prefer_h_1', 'person_prefer_h_2', 'person_prefer_h_3',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'contents_rn', 'contents_open_dt',
       'target', 'hour_band', 'person_prefer_d_1_n', 'person_prefer_d_1_s',
       'person_prefer_d_1_m', 'person_prefer_d_1_l', 'person_prefer_d_2_n',
       'person_prefer_d_2_s', 'person_prefer_d_2_m', 'person_prefer_d_2_l',
       'p

In [154]:
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]

train_data = train_data.drop(columns = cols_drop)
test_data = test_data.drop(columns = cols_drop)


# x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
# x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
# x_train.shape , y_train.shape , x_test.shape

In [155]:
# 이 부분을 추가한 모델이 성능이 조금은 더 높더라.
# 변수 개수가 여기에서 차이가 난다.
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_s" , "contents_attribute_d_s"),
    ("person_prefer_d_2_m" , "contents_attribute_d_m"),
    ("person_prefer_d_2_l" , "contents_attribute_d_l"),
    ("person_prefer_d_3_s" , "contents_attribute_d_s"),
    ("person_prefer_d_3_m" , "contents_attribute_d_m"),
    ("person_prefer_d_3_l" , "contents_attribute_d_l"),

    ("person_prefer_h_1_m" , "contents_attribute_h_m"),
    ("person_prefer_h_2_m" , "contents_attribute_h_m"),
    ("person_prefer_h_3_m" , "contents_attribute_h_m"),
    ("person_prefer_h_1_l" , "contents_attribute_h_l"),
    ("person_prefer_h_2_l" , "contents_attribute_h_l"),
    ("person_prefer_h_3_l" , "contents_attribute_h_l"),
]

    

In [156]:
bool_cols = ['d_l_match_yn',	'd_m_match_yn',	'd_s_match_yn'	,'h_l_match_yn','h_m_match_yn',	'h_s_match_yn',	'person_attribute_a']
def preprocess_data(
                    df:pd.DataFrame,
                    cols_equi:List[Tuple[str,str]]= [] )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()


    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

    return (df)

In [141]:
preprocess_data(train_data, cols_equi).columns

Index(['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2',
       'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'target', 'hour_band',
       'person_prefer_d_1_n', 'person_prefer_d_1_s', 'person_prefer_d_1_m',
       'person_prefer_d_1_l', 'person_prefer_d_2_n', 'person_prefer_d_2_s',
       'person_prefer_d_2_m', 'person_prefer_d_2_l', 'person_prefer_d_3_n',
       'person_prefer_d_3_s', 'person_prefer_d_3_m', 'person_prefer_d_3_

In [124]:
train_data = preprocess_data(train_data, cols_equi)
test_data = preprocess_data(test_data, cols_equi)

y_train = train_data.pop('target')
x_train = train_data
x_test = test_data

In [87]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()

In [88]:
cat_features

['person_attribute_a_1',
 'person_attribute_b',
 'person_prefer_c',
 'person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_e',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_i',
 'contents_attribute_a',
 'contents_attribute_j_1',
 'contents_attribute_c',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_m',
 'contents_attribute_e',
 'contents_attribute_h',
 'person_rn',
 'hour_band',
 'person_prefer_d_1_n',
 'person_prefer_d_1_s',
 'person_prefer_d_1_m',
 'person_prefer_d_1_l',
 'person_prefer_d_2_n',
 'person_prefer_d_2_s',
 'person_prefer_d_2_m',
 'person_prefer_d_2_l',
 'person_prefer_d_3_n',
 'person_prefer_d_3_s',
 'person_prefer_d_3_m',
 'person_prefer_d_3_l',
 'contents_attribute_d_n',
 'contents_attribute_d_s',
 'contents_attribute_d_m',
 'contents_attribute_d_l',
 'person_prefer_h_1_l',
 'person_prefer_h_1_m',
 'person_prefer_h_2_l',
 'person_prefer_h_2_m',
 'person_prefer_h_3_l',
 'person_

In [89]:
is_holdout = False
n_splits = 5 # 기존 5
iterations = 10000 # 기존 3000
patience = 200

cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

In [46]:
x_train['hour_band']

0         3.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
501946    3.0
501947    3.0
501948    2.0
501949    3.0
501950    3.0
Name: hour_band, Length: 501951, dtype: float64

In [63]:
import lightgbm as lgb
from sklearn.metrics import f1_score

scores = []
models = []

for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []
    
    model = lgb.LGBMClassifier(n_estimators = 3000)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            eval_metric = 'F1',
            early_stopping_rounds=patience ,
            verbose = 100
        )    
    y_pred = model.predict(x_train.iloc[tri])
    f1 = f1_score(y_train[tri], y_pred)
    print(f1)

    
    models.append(model)   

    if is_holdout:
        break    

[100]	valid_0's binary_logloss: 0.645452
[200]	valid_0's binary_logloss: 0.64154
[300]	valid_0's binary_logloss: 0.639518
[400]	valid_0's binary_logloss: 0.638075
[500]	valid_0's binary_logloss: 0.637177
[600]	valid_0's binary_logloss: 0.636678
[700]	valid_0's binary_logloss: 0.636119
[800]	valid_0's binary_logloss: 0.63557
[900]	valid_0's binary_logloss: 0.635047
[1000]	valid_0's binary_logloss: 0.634648
[1100]	valid_0's binary_logloss: 0.634356
[1200]	valid_0's binary_logloss: 0.634037
[1300]	valid_0's binary_logloss: 0.633789
[1400]	valid_0's binary_logloss: 0.633564
[1500]	valid_0's binary_logloss: 0.633315
[1600]	valid_0's binary_logloss: 0.633243
[1700]	valid_0's binary_logloss: 0.633105
[1800]	valid_0's binary_logloss: 0.633071
[1900]	valid_0's binary_logloss: 0.632891
[2000]	valid_0's binary_logloss: 0.632707
[2100]	valid_0's binary_logloss: 0.632578
[2200]	valid_0's binary_logloss: 0.632486
[2300]	valid_0's binary_logloss: 0.63243
[2400]	valid_0's binary_logloss: 0.632404
[250

In [66]:
pred = models[3].predict(x_test)

In [72]:
sum(pred)

25092

In [73]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission
print('target 1 개수 : ',sum(sample_submission['target']))
sample_submission.to_csv(f"{SUBMIT_PATH}lgbm.csv", index=False)

target 1 개수 :  25092


### category 로 변수를 바꾸고 나서 진행하면 성능이 좋아진다?
https://dataplay.tistory.com/2


In [157]:
train_data = preprocess_data(train_data, cols_equi)
test_data = preprocess_data(test_data, cols_equi)

In [158]:
for i in train_data.columns : 
    if i != 'target' :
        train_data[i] = train_data[i].astype('category') 
        test_data[i] = test_data[i].astype('category')



In [159]:
y_train = train_data.pop('target')
x_train = train_data
x_test = test_data

In [160]:
is_holdout = False
n_splits = 5 # 기존 5
iterations = 10000 # 기존 3000
patience = 200

cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

In [161]:
import lightgbm as lgb
from sklearn.metrics import f1_score

scores = []
models = []

for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []
    
    model = lgb.LGBMClassifier(n_estimators = 3000)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            eval_metric = 'F1',
            early_stopping_rounds=patience ,
            verbose = 100
        )    
    y_pred = model.predict(x_train.iloc[tri])
    f1 = f1_score(y_train[tri], y_pred)
    print(f1)

    
    models.append(model)   

    if is_holdout:
        break    

/home/isds/.conda/envs/isds/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/isds/.conda/envs/isds/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/isds/.conda/envs/isds/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/isds/.conda/envs/isds/lib/python3.8/site-packages/lig

[100]	valid_0's binary_logloss: 0.637838
[200]	valid_0's binary_logloss: 0.635776
[300]	valid_0's binary_logloss: 0.635257
[400]	valid_0's binary_logloss: 0.635593
[500]	valid_0's binary_logloss: 0.636043
0.7620066090542116
[100]	valid_0's binary_logloss: 0.640515
[200]	valid_0's binary_logloss: 0.638915
[300]	valid_0's binary_logloss: 0.638486
[400]	valid_0's binary_logloss: 0.638488
0.7538749582400996
[100]	valid_0's binary_logloss: 0.640144
[200]	valid_0's binary_logloss: 0.638385
[300]	valid_0's binary_logloss: 0.637962
[400]	valid_0's binary_logloss: 0.63821
0.7559572838195776
[100]	valid_0's binary_logloss: 0.641665
[200]	valid_0's binary_logloss: 0.639807
[300]	valid_0's binary_logloss: 0.639733
[400]	valid_0's binary_logloss: 0.640398
0.7488860223732349
[100]	valid_0's binary_logloss: 0.640807
[200]	valid_0's binary_logloss: 0.638739
[300]	valid_0's binary_logloss: 0.638754
[400]	valid_0's binary_logloss: 0.639264
0.7460315559270961


In [162]:
pred = models[0].predict(x_test)

In [163]:
sum(pred)

24692

In [164]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission
print('target 1 개수 : ',sum(sample_submission['target']))
sample_submission.to_csv(f"{SUBMIT_PATH}lgbm2.csv", index=False)

target 1 개수 :  24692


In [71]:
import lightgbm as lgb
from sklearn.metrics import f1_score

scores = []
models = []

def made_F1(y_true, y_pred) :
    return f1_score(y_true, y_pred, average='macro')

for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []
    
    model = lgb.LGBMClassifier(n_estimators = 300)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            eval_metric = made_F1,
            early_stopping_rounds=patience ,
            verbose = 100
        )    
    y_pred = model.predict(x_train.iloc[tri])
    f1 = f1_score(y_train[tri], y_pred)
    print(f1)

    
    models.append(model)   

    if is_holdout:
        break    

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
scores = []
models = []


models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations,
                               random_state=SEED,
                               task_type="GPU",
                               eval_metric="F1",
                               cat_features=cat_features,
                               one_hot_max_size=4,
                               bagging_temperature=0.2,
                               depth=10,
# depth default 값을 모르겠지만 학습이 엄청 오래걸리게 된다.
                               use_best_model=True)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            early_stopping_rounds=patience ,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break     # 0.6871

In [51]:
y_pred= clf.predict(x_test)

In [52]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

NameError: name 'y_test' is not defined

In [25]:
import numpy as np
import pandas as pd

X = x_train
y = y_train

import lightgbm as lgb
import time

start = time.time()

clf = lgb.LGBMClassifier()

lgbm = lightgbm.LGBMRegressor(boosting_type='gbdt',
                              max_depth=3,
                              n_estimators=3000,
                              learning_rate=0.1,
                              objective='Logloss',
                              n_jobs=-1, random_state=0)

from sklearn.model_selection import cross_val_score

lgbm_scores = cross_val_score(lgbm, X, y,
                              scoring = 'Logloss',
                              #scoring="neg_mean_squared_error",
                              cv=5, n_jobs=1)

lgbm_rmse = np.sqrt(-lgbm_scores)
print("Cross-validated RMSE for LightGBM: ", np.mean(lgbm_rmse))

end = time.time()
diff = end - start
print('Execution time for LightGBM (in Seconds):', diff)

ValueError: 'Logloss' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.

In [108]:
scores = []
models = []


models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations,
                               random_state=SEED,
                               task_type="GPU",
                               eval_metric="F1",
                               cat_features=cat_features,
                               one_hot_max_size=4,
                               bagging_temperature=0.2,
                               depth=10,
# depth default 값을 모르겠지만 학습이 엄청 오래걸리게 된다.
                               use_best_model=True)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            early_stopping_rounds=patience ,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break     # 0.6871

Learning rate set to 0.016489
0:	learn: 0.6208467	test: 0.6196185	best: 0.6196185 (0)	total: 104ms	remaining: 17m 19s
100:	learn: 0.6514052	test: 0.6550750	best: 0.6550750 (100)	total: 8.94s	remaining: 14m 35s
200:	learn: 0.6577890	test: 0.6607306	best: 0.6608856 (196)	total: 17.8s	remaining: 14m 28s
300:	learn: 0.6662070	test: 0.6724514	best: 0.6724514 (300)	total: 26.4s	remaining: 14m 9s
400:	learn: 0.6737380	test: 0.6790632	best: 0.6790632 (400)	total: 34.8s	remaining: 13m 53s
500:	learn: 0.6794737	test: 0.6842533	best: 0.6842950 (499)	total: 43.2s	remaining: 13m 39s
600:	learn: 0.6848177	test: 0.6863817	best: 0.6863817 (600)	total: 51.6s	remaining: 13m 26s
700:	learn: 0.6898518	test: 0.6877063	best: 0.6877359 (693)	total: 60s	remaining: 13m 15s
800:	learn: 0.6948310	test: 0.6885246	best: 0.6886186 (770)	total: 1m 8s	remaining: 13m 5s
900:	learn: 0.6992347	test: 0.6884749	best: 0.6886186 (770)	total: 1m 16s	remaining: 12m 53s
1000:	learn: 0.7034139	test: 0.6882876	best: 0.6886197 (9

In [109]:
test_data = preprocess_data(test_data, cols_equi)
x_test = test_data

In [110]:
print(scores)
print(np.mean(scores))

# hyperparemeter 수정 X
# 결과 0.75 이전꺼

[0.6886734849641276, 0.6876681337275069, 0.6859243202488102, 0.681712553433618, 0.6822915660322675]
0.685254011681266


In [111]:
max_score = 0 
for threshold in [0.38, 0.39, 0.40, 0.41, 0.42,] :
    pred_list = []
    scores = []
    for i,(tri, vai) in enumerate( cv.split(x_train) ):
        pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
        pred = np.where(pred >= threshold , 1, 0)
        score = f1_score(y_train[vai],pred)
        scores.append(score)
        pred = models[i].predict_proba(x_test)[:, 1]
        pred_list.append(pred)
    print(scores)
    print('threshold', threshold, 'mean', np.mean(scores))
    if np.mean(scores) > max_score : 
        max_score = np.mean(scores)
        fin_threshold = threshold
print("max score : ", max_score)
print("fin_threshold : ", fin_threshold)

# 최종모델 threshold 선정 후 다시 pred_list 만들기.
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= fin_threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)

[0.7184832593787818, 0.7151081897999327, 0.7119947362997139, 0.7136985421617108, 0.7150238765764664]
threshold 0.38 mean 0.7148617208433211
[0.7174931062052312, 0.714844634138455, 0.7120086548864045, 0.7134181983503279, 0.7147185825510448]
threshold 0.39 mean 0.7144966352262927
[0.7169063946723442, 0.7151116671982206, 0.711843523794856, 0.7127521412484178, 0.7140411959689069]
threshold 0.4 mean 0.7141309845765491
[0.7159854715484489, 0.7146972631997217, 0.7110334473677734, 0.7110486065457814, 0.7129343401070964]
threshold 0.41 mean 0.7131398257537644
[0.7147870352362188, 0.7125030379725619, 0.7099969255569312, 0.7098220682563363, 0.711525467219775]
threshold 0.42 mean 0.7117269068483646
max score :  0.7148617208433211
fin_threshold :  0.38


In [112]:
# hard voting 방법으로 변환한 앙상블

count = [0] * len(pred_list[0])
for i,pred in enumerate(pred_list) :
    for  idx, data in enumerate(pred) :
        if data >= threshold :
            count[idx] += 1
pred = np.where(np.array(count) >= 3, 1, 0)


In [113]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission
print('target 1 개수 : ',sum(sample_submission['target']))

target 1 개수 :  33121


In [114]:
sample_submission.to_csv(f"{SUBMIT_PATH}catboost-made4-version2.csv", index=False)

In [115]:
# 위와 똑같은 것을 기존 산술평균 앙상블로 제출한다면?

threshold = 0.39

In [116]:
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

[0.7174931062052312, 0.714844634138455, 0.7120086548864045, 0.7134181983503279, 0.7147185825510448]
0.7144966352262927


In [117]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

In [118]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission
print('target 1 개수 : ',sum(sample_submission['target']))

target 1 개수 :  34371


In [119]:
sample_submission.to_csv(f"{SUBMIT_PATH}catboost-made4-version3.csv", index=False)